# Imports, Utils

In [1]:
import pystore
import talib as ta
import plotly.graph_objects as go
import pandas as pd
import bt
import plotly.express as px
import plotly.graph_objects as go
import datetime

def merge_timeseries(datasets, column):
    dfs = []
    for asset, df in datasets.items():
        dfs.append(df[column].rename(asset))
    merged = pd.concat(dfs, axis=1)
    # if column == 'close':
    merged.fillna(inplace=True, method='ffill')
    return merged
    # Outter join
    # ffill 
    #backfill na?
# Resampling
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html
# Default gives you mean values, need to adapt for candles.

# Merging

# Time Shifts

# Rolling (Moving Average) and Expanding (Average since beginning)
# Can input column or entire df, and periods

# Exponentially Weighted Moving Average (ewm)
# Can input column or entire df, and periods

# Create time series date time index from two dates

# Bolinger Bands
# 20 SMA
# Upper = 20 SMA + 2x 20 StDev
# Lower = 20 SMA - 2x 20 StDev
# 

# Statistics
# https://mrjbq7.github.io/ta-lib/func_groups/statistic_functions.html
# Beta, Correaltion (r), Linear Regression (regression, intercept, slope, angle), Standard Deviation, Variance, Time Series Forecast



# Read daily data from database into memory

In [2]:


storage_dir = '/Volumes/CAPA/.storage'
data = {}
chart_data = {}
pystore.set_path(storage_dir)
store = pystore.store('historical')
collections = store.list_collections()
collection = store.collection('Crypto.Candles')
items = collection.list_items()
all_symbols = []
for s in items:
    all_symbols.append(s + "/Day")

for symbol in all_symbols:
    data[symbol] = collection.item(symbol).to_pandas()
data_length = len(data.keys())
print(data_length)

296


# Set indicator functions

In [3]:
indicators = [
    {'label': 'Close', 
    'value':'close'},
    {'label': 'Scaled Close',
    'value':'scaled_close'},    
    # {'label': 'Rebase',
    # 'value':'rebase'},
    {'label': 'Volume',
    'value':'volume'},
    {'label': '30 Day SMA - Close',
    'value':'sma_30_close'},
    {'label': '60 Day SMA - Close',
    'value':'sma_60_close'},
    {'label': '90 Day SMA - Close',
    'value':'sma_90_close'},
    {'label': '30 Day SMA - Scaled Close',
    'value':'sma_30_scaled_close'},
    {'label': '60 Day SMA - Scaled Close',
    'value':'sma_60_scaled_close'},
    {'label': '90 Day SMA - Scaled Close',
    'value':'sma_90_scaled_close'},
]

def calc_scaled_close(datasets):
    for sym in datasets:
        value = datasets[sym]["close"] / max(datasets[sym]["close"])
        data[sym]["scaled_close"] = value
    return datasets

def calc_changes(datasets):
    for sym in datasets:
        data[sym]["1d_change_pct"] = datasets[sym]["close"].pct_change()
        data[sym]["3day_change_pct"] = datasets[sym]["close"].pct_change(3)
        data[sym]["7day_change_pct"] = datasets[sym]["close"].pct_change(7)
        data[sym]["30day_change_pct"] = datasets[sym]["close"].pct_change(30)
        data[sym]["3m_change_pct"] = datasets[sym]["close"].pct_change(90)
        data[sym]["6m_change_pct"] = datasets[sym]["close"].pct_change(180)
        data[sym]["1y_change_pct"] = datasets[sym]["close"].pct_change(365)
        data[sym]["2y_change_pct"] = datasets[sym]["close"].pct_change(730)
        data[sym]["1d_change"] = datasets[sym]["close"].diff()
        data[sym]["3d_change"] = datasets[sym]["close"].diff(3)
        data[sym]["7day_change"] = datasets[sym]["close"].diff(7)
        data[sym]["30day_change"] = datasets[sym]["close"].diff(30)
        data[sym]["3m_change"] = datasets[sym]["close"].diff(90)
        data[sym]["6m_change"] = datasets[sym]["close"].diff(180)
        data[sym]["1y_change"] = datasets[sym]["close"].diff(365)
        data[sym]["2y_change"] = datasets[sym]["close"].diff(730)
    return datasets

def calc_sma(datasets, column, timeperiod):
    for sym in datasets:
        col_array = datasets[sym][column].to_numpy()
        data[sym]["{}-sma-{}".format(column, timeperiod)] = ta.SMA(col_array,timeperiod)
    return datasets

def calc_rsi(datasets, timeperiod):
    for sym in datasets:
        col_array = datasets[sym]['close'].to_numpy()
        data[sym]["rsi-{}".format(timeperiod)] = ta.RSI(col_array,timeperiod)
    return datasets

def calc_bb(datasets, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0):
    for sym in datasets:
        col_array = datasets[sym]['close'].to_numpy()
        upperband, middleband, lowerband = ta.BBANDS(col_array, timeperiod, nbdevup, nbdevdn, matype)
        data[sym]["bb-{}-{}-{}-upper".format(timeperiod, nbdevup, nbdevup)] = upperband
        data[sym]["bb-{}-{}-{}-middle".format(timeperiod, nbdevup, nbdevup)] = middleband
        data[sym]["bb-{}-{}-{}-lower".format(timeperiod, nbdevup, nbdevup)] = lowerband
    return datasets

def calc_basic_stats(datasets):
    calc_scaled_close(data)
    calc_changes(data)
    calc_sma(data, "close", 30)
    calc_sma(data, "close", 60)
    calc_sma(data, "close", 90)
    calc_sma(data, "scaled_close", 30)
    calc_sma(data, "scaled_close", 60)
    calc_sma(data, "scaled_close", 90)
    calc_sma(data, "volume", 30)
    calc_sma(data, "volume", 60)
    calc_sma(data, "volume", 90)
    calc_rsi(data, 7)
    calc_rsi(data, 14)
    calc_rsi(data, 28)
    calc_bb(data)


# Run Indicators

In [4]:
calc_basic_stats(data)
data["BTC/Day"]

,open,high,low,close,volume,scaled_close,1d_change_pct,3day_change_pct,7day_change_pct,30day_change_pct,...,scaled_close-sma-90,volume-sma-30,volume-sma-60,volume-sma-90,rsi-7,rsi-14,rsi-28,bb-5-2-2-upper,bb-5-2-2-middle,bb-5-2-2-lower
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-10-19,3812.004225,5693.210514,3806.381676,5137.927269,2.824693,0.080812,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-10-20,5137.927269,5998.207831,5137.927269,5698.297439,3.455172,0.089626,0.109065,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-10-21,5698.297439,6660.000000,5692.000000,5985.000000,0.638032,0.094135,0.050314,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-10-22,5985.000000,6660.000000,5692.000000,6500.000000,1.873153,0.102235,0.086048,0.265102,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-10-23,6229.650000,6500.000000,5692.000000,6150.000000,2.809644,0.096731,-0.053846,0.079270,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6811.195613,5894.244942,4977.294270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-01,37257.600000,37898.900000,35698.100000,36689.400000,7683.095028,0.577071,-0.015251,0.060381,-0.042575,-0.351416,...,0.828202,9301.150327,7374.068235,6279.327469,41.219242,37.536253,39.341560,37815.157168,35970.560000,34125.962832
2021-06-02,36694.600000,38230.700000,35926.800000,37589.000000,6046.543408,0.591220,0.024519,0.054556,-0.042109,-0.342386,...,0.826317,9343.353978,7432.974106,6306.242653,46.267323,39.858195,40.458970,38553.124369,36356.120000,34159.115631
2021-06-03,37589.000000,39474.500000,37176.200000,39249.900000,5991.138107,0.617344,0.044186,0.053474,0.018745,-0.262169,...,0.824655,9307.073313,7498.340630,6331.729237,54.655320,43.997340,42.487458,39652.542769,37286.060000,34919.577231


# Generate Some Charts

In [5]:
## Simple line graph that plots two values

In [6]:

btc_df = data['BTC/Day']
fig = px.scatter(btc_df, y=['close'])
volume_scatter = go.Scatter(x=btc_df.index, y=btc_df['volume'], mode='lines', name='Volume')
close_scatter = go.Scatter(x=btc_df.index, y=btc_df['close'], mode='lines', name='Close')
overlay_fig = go.FigureWidget()
overlay_fig.add_trace(close_scatter)
overlay_fig.add_trace(volume_scatter)
overlay_fig.show()

In [7]:
## Line graph widget using dropdown menus to select what is graphed.

In [8]:
df = data['BTC/Day']
fig = go.FigureWidget()
buttons = []
visible = {}
# Add Traces
all_cols = list(df.keys())

for col in df.keys():
    visible[col] = False

for col in df.keys():
    current_vis = visible.copy()
    current_vis[col] = True
    fig.add_trace(
        go.Scatter(x=list(df.index),
                y=list(df[col]),
                name=col,
                line=dict(color="#33CFA5")))
    button = {
        'label' : col,
        'method' : "update",
        'args' : [
            {"visible": list(current_vis.values())},
            {"title": "BTC {}".format(col)}
            ],
    }
    buttons.append(button)


fig.update_layout(updatemenus=[
        dict(
            active=0,
            buttons=buttons,
        )
    ])

# Set title
fig.update_layout(title_text="BTC", width=1000)
fig.show()

# Backtests


# Backtest Utils and Config

## Select Where
Pass a function
Examples:
- Select all that went up, went up x periods in a row, went up x% over x periods
- Select all with property (transactions, marketcap) > number
- Select based on crossover of two things, up or down
- Select top x items or percent based on property

In [9]:
closing_prices = merge_timeseries(data, 'close')

backtest_frequencies = {
    'once' : bt.algos.RunOnce(),
    'weekly' : bt.algos.RunWeekly(),
    'monthly' : bt.algos.RunMonthly(),
    'quarterly' : bt.algos.RunQuarterly(),
    'yearly' : bt.algos.RunYearly()
}

backtest_weightings = {
    'equal' : bt.algos.WeighEqually(),
    'inv-vol' : bt.algos.WeighInvVol(),
    'mean-var' : bt.algos.WeighMeanVar(),
    'erc' : bt.algos.WeighERC(),
}

def create_strategies():
    all_strats = []
    for freq in backtest_frequencies.keys():
        for weight in backtest_weightings.keys():
            all_strats.append(bt.Strategy(
                "{}-{}".format(freq, weight),
                [
                    backtest_frequencies[freq],
                    bt.algos.SelectHasData(),
                    backtest_weightings[weight],
                    bt.algos.Rebalance()
                ]
                )
            )
    return all_strats

def create_backtests(dataset):
    all_strats = create_strategies()
    all_bts = []
    for strat in all_strats:
        all_bts.append(bt.Backtest(strat, dataset))
    return all_bts

def run_all_backtests(dataset):
    all_bts = create_backtests(dataset)
    res = bt.run(*all_bts)
    res.display()


In [10]:
run_all_backtests(bench_closing_prices)

NameError: name 'bench_closing_prices' is not defined


# Benchmark Config

In [52]:
bench = bt.Strategy("Long BTC/ETH Equal", [
    bt.algos.RunOnce(),
    bt.algos.SelectAll(),
    bt.algos.WeighEqually(),
    bt.algos.Rebalance()
    ]
)
start_date = datetime.date(2018, 1, 1)
end_date = datetime.date(2021, 6, 1)
date_index = pd.date_range(start_date,end_date, freq='D')
date_index.rename("timestamp", inplace=True)
columns = ['usdc']
usdc_price_df = pd.DataFrame(index=date_index, columns=columns)
usdc_coupon_df = pd.DataFrame(index=date_index, columns=columns)

# usdc_empty_df['timestamp'] = usdc_empty_df.index
usdc_price_df.fillna(1, inplace=True)
usdc_coupon_df.fillna(.1, inplace=True)
usdc = bt.core.CouponPayingSecurity('usdc')
usdc.setup(usdc_price_df, coupons=usdc_coupon_df)
# for date in date_index:
#     usdc.update(date, ) 
# delta = datetime.timedelta(days=1)
bench_data = {}
bench_data['BTC'] = data["BTC/Day"]
bench_data['ETH'] = data["ETH/Day"]
bench_data['LINK'] = data["LINK/Day"]
bench_closing_prices = merge_timeseries(bench_data, 'close')

test_interest = bt.FixedIncomeStrategy("Interest Test",
[
    bt.algos.Run,
    bt.algos.SelectAll(),
    bt.algos.WeighEqually(),
    bt.algos.Rebalance()
])

# while start_date <= end_date:
#     usdc.update(start_date, data={'coupon':.1})
#     start_date += delta
print(usdc.data)

test_bt = bt.Backtest(test_interest, usdc_price_df)
test_res = bt.run(test_bt)
test_res.display()

test_bench = bt.Backtest(bench, bench_closing_prices)

            value  position  notional_value  outlay  coupon  holding_cost
timestamp                                                                
2018-01-01    0.0       0.0             0.0     0.0     0.0           0.0
2018-01-02    0.0       0.0             0.0     0.0     0.0           0.0
2018-01-03    0.0       0.0             0.0     0.0     0.0           0.0
2018-01-04    0.0       0.0             0.0     0.0     0.0           0.0
2018-01-05    0.0       0.0             0.0     0.0     0.0           0.0
...           ...       ...             ...     ...     ...           ...
2021-05-28    0.0       0.0             0.0     0.0     0.0           0.0
2021-05-29    0.0       0.0             0.0     0.0     0.0           0.0
2021-05-30    0.0       0.0             0.0     0.0     0.0           0.0
2021-05-31    0.0       0.0             0.0     0.0     0.0           0.0
2021-06-01    0.0       0.0             0.0     0.0     0.0           0.0

[1248 rows x 6 columns]


NotImplementedError: RunPeriod Algo is an abstract class!

# Equal Weight Rebalancing

In [ ]:
test_data = {}

test_data['ETH/Day'] = data['ETH/Day']
test_data['BTC/Day'] = data['BTC/Day']
test_data['LINK/Day'] = data['LINK/Day']

s1 = bt.Strategy('Weekly Equal', [
    bt.algos.RunWeekly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighEqually(),
    bt.algos.Rebalance()
    ]
)

s2 = bt.Strategy('Monthly Equal', [
    bt.algos.RunMonthly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighEqually(),
    bt.algos.Rebalance()
    ]
)

s3 = bt.Strategy('Quarterly Equal', [
    bt.algos.RunQuarterly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighEqually(),
    bt.algos.Rebalance()
    ]
)

s4 = bt.Strategy('Yearly Equal', [
    bt.algos.RunYearly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighEqually(),
    bt.algos.Rebalance()
    ]
)

# test0 = bt.Backtest(s0, closing_prices)
test1 = bt.Backtest(s1, closing_prices)
test2 = bt.Backtest(s2, closing_prices)
test3 = bt.Backtest(s3, closing_prices)
test4 = bt.Backtest(s4, closing_prices)
res1 = bt.run(test1, test2, test3, test4, test_bench)
res1.display()

In [ ]:
merge_timeseries(data, 'close')['ETH/Day']